In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Import Data

In [5]:
data = pd.read_csv('data/cumulative.csv')
data

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,9560,10031643,K07984.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,0,0,...,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
9560,9561,10090151,K07985.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,...,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9561,9562,10128825,K07986.01,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,...,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9562,9563,10147276,K07987.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.021,0,0,1,...,-236.0,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


## Checking NaN in columns

In [4]:
data.isnull().sum()

rowid                   0
kepid                   0
kepoi_name              0
kepler_name          7270
koi_disposition         0
koi_pdisposition        0
koi_score            1510
koi_fpflag_nt           0
koi_fpflag_ss           0
koi_fpflag_co           0
koi_fpflag_ec           0
koi_period              0
koi_period_err1       454
koi_period_err2       454
koi_time0bk             0
koi_time0bk_err1      454
koi_time0bk_err2      454
koi_impact            363
koi_impact_err1       454
koi_impact_err2       454
koi_duration            0
koi_duration_err1     454
koi_duration_err2     454
koi_depth             363
koi_depth_err1        454
koi_depth_err2        454
koi_prad              363
koi_prad_err1         363
koi_prad_err2         363
koi_teq               363
koi_teq_err1         9564
koi_teq_err2         9564
koi_insol             321
koi_insol_err1        321
koi_insol_err2        321
koi_model_snr         363
koi_tce_plnt_num      346
koi_tce_delivname     346
koi_steff   

## Del useless columns like ID and names. Also del columns 'koi_teq_err1' and 'koi_teq_err2', because they consist only of NaN. Then drop missing data.

In [5]:
del data['koi_teq_err1']
del data['koi_teq_err2']
del data['kepler_name']
del data['koi_tce_delivname']
del data['rowid']
del data['kepid']
del data['koi_pdisposition']
data = data.dropna()
data

,kepoi_name,koi_disposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,K00752.01,CONFIRMED,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,K00752.02,CONFIRMED,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,K00753.01,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,K00754.01,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,K00755.01,CONFIRMED,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,K07984.01,FALSE POSITIVE,0.000,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,...,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
9560,K07985.01,FALSE POSITIVE,0.000,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,...,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9561,K07986.01,CANDIDATE,0.497,0,0,0,0,1.739849,1.780000e-05,-1.780000e-05,...,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9562,K07987.01,FALSE POSITIVE,0.021,0,0,1,0,0.681402,2.434000e-06,-2.434000e-06,...,-236.0,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


## Extract feautures and targets from data

In [6]:
candidates = data[data.koi_disposition == 'CANDIDATE']
X_candidates = candidates.iloc[:,2:]
X = data[data.koi_disposition != 'CANDIDATE'].iloc[:,2:]

y = np.array(data[data.koi_disposition != 'CANDIDATE'].iloc[:,1])
y[np.where(y == 'CONFIRMED')] = 1
y[np.where(y == 'FALSE POSITIVE')] = 0
y = y.astype('int')

## Split X and y to X_test/train and y_test/train

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_candidates = scaler.transform(X_candidates)

## MLPClassifier

In [8]:
clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=100, alpha=0.0001,
                     solver='adam', verbose=10,activation = 'logistic',  random_state=21,tol=0.000000001).fit(X_train, y_train)

Iteration 1, loss = 0.66763778
Iteration 2, loss = 0.65506298
Iteration 3, loss = 0.61122597
Iteration 4, loss = 0.43693843
Iteration 5, loss = 0.21537866
Iteration 6, loss = 0.11251564
Iteration 7, loss = 0.06818150
Iteration 8, loss = 0.04737717
Iteration 9, loss = 0.03910862
Iteration 10, loss = 0.03276233
Iteration 11, loss = 0.03061306
Iteration 12, loss = 0.02827920
Iteration 13, loss = 0.02611003
Iteration 14, loss = 0.02489574
Iteration 15, loss = 0.02387608
Iteration 16, loss = 0.02296440
Iteration 17, loss = 0.02242973
Iteration 18, loss = 0.02203388
Iteration 19, loss = 0.02144443
Iteration 20, loss = 0.02178439
Iteration 21, loss = 0.02103392
Iteration 22, loss = 0.02237272
Iteration 23, loss = 0.02299687
Iteration 24, loss = 0.02032154
Iteration 25, loss = 0.02029863
Iteration 26, loss = 0.02033092
Iteration 27, loss = 0.02071047
Iteration 28, loss = 0.02116875
Iteration 29, loss = 0.02139459
Iteration 30, loss = 0.02022596
Iteration 31, loss = 0.02030103
Iteration 32, los

In [9]:
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1508
           1       1.00      0.98      0.99       905

    accuracy                           0.99      2413
   macro avg       0.99      0.99      0.99      2413
weighted avg       0.99      0.99      0.99      2413

0.9921259842519685


In [10]:
df_predictions = pd.DataFrame({'Name': candidates['kepoi_name'],
                               'Prediction': clf.predict(X_candidates)})
df_predictions

,Name,Prediction
37,K00760.01,1
58,K00777.01,1
62,K00780.02,1
63,K00115.03,1
72,K00799.01,1
...,...,...
9550,K07888.01,1
9551,K07889.01,1
9557,K03875.01,0
9558,K03891.01,1


## Keras.Sequential

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [12]:
model = Sequential()
model.add(Dense(4221, activation='relu', input_dim=41))
model.add(Dense(2000, activation = 'relu'))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(500, activation = 'relu'))
model.add(Dense(250, activation = 'relu'))
model.add(Dense(125, activation = 'relu'))
model.add(Dense(60, activation = 'relu'))

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(x=X_train,y=y_train, epochs=5)

Epoch 1/5
114/114 [==============================] - 10s 77ms/step - loss: 0.2141 - accuracy: 0.9306
Epoch 2/5
114/114 [==============================] - 8s 74ms/step - loss: 0.0661 - accuracy: 0.9901
Epoch 3/5
114/114 [==============================] - 9s 76ms/step - loss: 0.0193 - accuracy: 0.9951
Epoch 4/5
114/114 [==============================] - 9s 75ms/step - loss: 0.0539 - accuracy: 0.9932
Epoch 5/5
114/114 [==============================] - 9s 78ms/step - loss: 0.0385 - accuracy: 0.9912


In [13]:
y_pred = model.predict_classes(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1508
           1       1.00      0.98      0.99       905

    accuracy                           0.99      2413
   macro avg       0.99      0.99      0.99      2413
weighted avg       0.99      0.99      0.99      2413

0.9912971404890178


In [14]:
df_test = pd.DataFrame()
df_test['y_test']= y_test
df_test['y_pred'] = y_pred
df_test

,y_test,y_pred
0,1,1
1,0,0
2,1,1
3,0,0
4,0,0
...,...,...
2408,0,0
2409,0,0
2410,0,0
2411,0,0


## Make precisions for 'candidates'

In [15]:
df_predictions['Prediction by keras'] = model.predict_classes(X_candidates)
df_predictions

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


,Name,Prediction,Prediction by keras
37,K00760.01,1,1
58,K00777.01,1,1
62,K00780.02,1,1
63,K00115.03,1,1
72,K00799.01,1,1
...,...,...,...
9550,K07888.01,1,1
9551,K07889.01,1,1
9557,K03875.01,0,1
9558,K03891.01,1,1
